In [74]:
import os
import pandas as pd
import numpy as np
import datetime
import scipy
from scipy.stats import skew, kurtosis
import tensorflow as tf

In [2]:
# Read Accelerometer Data
acc_data = pd.read_csv('../data/all_accelerometer_data_pids_13.csv')


def get_time_value(x):
  # x is ms. it is divided by 1000 to get microsecond
  t = datetime.datetime.fromtimestamp(x/1000.0)
  t = t.replace(microsecond = 0)
  return int(t.timestamp())

acc_data['window10'] = acc_data['time'].apply(get_time_value)
acc_data = acc_data.drop(columns="time")
acc_data = acc_data.rename(columns = {"window10": "time"})

acc_data.head()

,pid,x,y,z,time
0,JB3156,0.0000,0.0000,0.0000,0
1,CC6740,0.0000,0.0000,0.0000,0
2,SA0297,0.0758,0.0273,-0.0102,1493733882
3,SA0297,-0.0359,0.0794,0.0037,1493733882
4,SA0297,-0.2427,-0.0861,-0.0163,1493733882


In [3]:
acc_data['pid'].unique()

array(['JB3156', 'CC6740', 'SA0297', 'PC6771', 'BK7610', 'DC6359',
       'MC7070', 'MJ8002', 'BU4707', 'JR8022', 'HV0618', 'SF3079',
       'DK3500'], dtype=object)

In [4]:
# Read clean tac data for pid = BK7610
clean_tac_data = pd.read_csv('../data/clean_tac/BK7610_clean_TAC.csv')
# clean_tac_data = pd.read_csv('../data/clean_tac/JB3156_clean_TAC.csv')
clean_tac_data["tac"] = np.where(clean_tac_data["TAC_Reading"] > 0.08, 1, 0)
clean_tac_data = clean_tac_data.drop(columns="TAC_Reading")
clean_tac_data = clean_tac_data.rename(columns={"tac": "TAC_Reading"})
clean_tac_data.describe()


,timestamp,TAC_Reading
count,5.700000e+01,57.000000
mean,1.493758e+09,0.228070
std,2.841595e+04,0.423318
min,1.493719e+09,0.000000
25%,1.493729e+09,0.000000
50%,1.493756e+09,0.000000
75%,1.493782e+09,0.000000
max,1.493808e+09,1.000000


In [5]:
# Filtering for specific PID (temps)
acc_data_pid = acc_data[acc_data.pid == "BK7610"]
acc_data_pid['pid'].unique()

array(['BK7610'], dtype=object)

In [6]:
acc_data_pid.describe()

,x,y,z,time
count,1.225727e+06,1.225727e+06,1.225727e+06,1.225727e+06
mean,-6.497703e-03,7.507374e-03,2.747567e-03,1.493752e+09
std,1.380473e-01,1.387602e-01,1.279124e-01,9.276766e+03
min,-4.274800e+00,-6.948900e+00,-5.277200e+00,1.493736e+09
25%,-9.400000e-03,-6.000000e-03,-7.300000e-03,1.493744e+09
50%,1.000000e-04,1.000000e-04,5.700000e-03,1.493752e+09
75%,8.300000e-03,9.400000e-03,1.140000e-02,1.493760e+09
max,6.450300e+00,5.344100e+00,4.656500e+00,1.493768e+09


In [7]:
acc_data_pid.shape

(1225727, 5)

In [8]:
# Up sampling tac data to match acc data
clean_ts = clean_tac_data['timestamp'] 
acc_ts = acc_data_pid['time']
all_labels = list()
offset_tac, offset_acc = 0, 0
# print(acc_ts.iloc[0])
# print(clean_ts.loc[0])
# print(clean_tac_data.loc[0]['TAC_Reading'])
# # acc_ts.iloc[0] #1493735870653
while offset_tac < len(clean_ts) and offset_acc < len(acc_ts):
  
  while acc_ts.iloc[offset_acc] < clean_ts.iloc[offset_tac]:
    all_labels.append([clean_tac_data.iloc[offset_tac]['TAC_Reading'], acc_ts.iloc[offset_acc]])
    offset_acc += 1
    if offset_acc >= len(acc_ts):
      break

  offset_tac += 1

all_labels


[[0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735870],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735871],
 [0, 1493735872],
 [0, 14937

In [9]:
all_labels_df = pd.DataFrame(all_labels, columns = ["tac", "time"])
all_labels_df.shape, acc_data_pid.shape
# merged = merged.drop_duplicates().reset_index(drop=True)
# merged.to_csv("../data/BK7610_final_final.csv")

((1225727, 2), (1225727, 5))

In [10]:
# merged = acc_data_pid.head(10).merge(all_labels_df.head(10), on = 'time', how='inner')
# merged['time'].value_counts()

In [11]:
# len(all_labels_df['time'].unique()), len(acc_data_pid['time'].unique())

In [12]:
# acc_data_pid['tac_reading'] = 
# TODO: Make sure tac data is sorted on timestamp
clean_tac_data["from"] = clean_tac_data["timestamp"].shift(1, fill_value=-1) + 1

In [13]:
clean_tac_data.index = pd.IntervalIndex.from_arrays(clean_tac_data["from"], clean_tac_data["timestamp"], closed = "both")

In [14]:
acc_data_pid['tac'] = acc_data_pid["time"].apply(lambda x: clean_tac_data.iloc[clean_tac_data.index.get_loc(x)]["TAC_Reading"])
acc_data_pid

/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_92662/2638206493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_data_pid['tac'] = acc_data_pid["time"].apply(lambda x: clean_tac_data.iloc[clean_tac_data.index.get_loc(x)]["TAC_Reading"])


,pid,x,y,z,time,tac
47136,BK7610,0.1261,-0.0078,-0.0243,1493735870,0
47138,BK7610,0.1336,-0.0697,-0.0446,1493735870,0
47140,BK7610,0.1443,-0.0474,-0.0447,1493735870,0
47142,BK7610,0.1255,-0.0038,0.0111,1493735870,0
47144,BK7610,0.1076,0.0032,0.0276,1493735870,0
...,...,...,...,...,...,...
6071104,BK7610,-0.0784,-0.0161,0.1719,1493767770,1
6071108,BK7610,-0.0395,-0.0816,0.1634,1493767770,1
6071112,BK7610,0.0160,-0.0853,0.0906,1493767770,1
6071117,BK7610,0.0901,-0.0767,0.0162,1493767770,1


In [15]:
# print(acc_data_pid["tac"].unique().sort())
# clean_tac_data
# min: 1,493,718,714
# max: 1,493,807,899
# 
# acc_data_pid
# min: 1,493,735,870
# max: 1,493,767,770
# acc_data_pid["time"].max()
# groups = acc_data_pid.groupby(["time"])
# print(groups.apply(lambda x: x["tac"]<20))
# time_stamps = groups.apply(lambda x: x["tac"]<20)
# time_stamps 

min(acc_data_pid.index), max(acc_data_pid.index)

(47136, 6071121)

In [16]:
# TODO: Make n = 10 after either removing one record which has 7 records for a second or by adding 3 dummy values to it (latter is better)
# frame_temp.groupby([ "pid", "window10"]).count().describe()
# We are sampling with replacement, which should be okay since it is within a second
acc_data_pid_20s = acc_data_pid.groupby([ "pid", "time"]).sample(n = 20, replace=True)

In [17]:
# TODO: Make this as an assert statement in the begininng for both tac and accelerometer data
acc_data_pid_20s["time"].is_monotonic_increasing

True

In [18]:
acc_data_sliding = acc_data_pid_20s.copy()
cols = ["x", "y", "z"]
window_size = 10 # including current

for col in cols:
    cols_to_append = []
    for i in range(0, window_size):
        shifted_col_name = str(col) + "_" + str(i)
        acc_data_sliding[shifted_col_name] = acc_data_sliding[col].shift(i, fill_value = 0)
        cols_to_append.append(shifted_col_name)
    
    # we have (windpw_size ) columsn for col
    # Uncomment to keep original columns
    acc_data_sliding = acc_data_sliding.drop(columns=[col])    
    acc_data_sliding[str(col)] = acc_data_sliding[cols_to_append].values.tolist()
    
    # acc_data_sliding = acc_data_sliding.drop(columns=cols_to_append)

In [133]:

# # x_sliding_window.shape
pids = ["BK7610"]
final = []
labels = []
for pid in pids:
  # temptemp = acc_data_pid_20s[acc_data_pid_20s['pid'] == pid]
  temptemp = acc_data_sliding[acc_data_sliding['pid'] == pid]
  times = temptemp.time.unique()
  final_temp =[]
  labels_temp = []
  for time in times:
    # x = np.lib.stride_tricks.sliding_window_view(frame_temp2[frame_temp2.pid == pid and frame_temp2.window10 == time], window_shape = 10)
    temptemptemp = temptemp[temptemp['time'] == time]
    # TODO: Create x y z sliding windows
    # x = np.lib.stride_tricks.sliding_window_view(temptemptemp["x"], window_shape = 10)
    # y = np.lib.stride_tricks.sliding_window_view(temptemptemp["y"], window_shape = 10)
    # z = np.lib.stride_tricks.sliding_window_view(temptemptemp["z"], window_shape = 10)
    x_dash = np.array(temptemptemp[["x_"+str(i) for i in range(window_size)]]).flatten()
    y_dash = np.array(temptemptemp[["y_"+str(i) for i in range(window_size)]]).flatten()
    z_dash = np.array(temptemptemp[["z_"+str(i) for i in range(window_size)]]).flatten()
    # a = np.vstack((temptemptemp["x"].apply(lambda x: np.array(x, dtype="float32")), temptemptemp["y"].apply(lambda x: np.array(x, dtype="float32")), temptemptemp["z"].apply(lambda x: np.array(x, dtype="float32"))))
    a = np.transpose(np.vstack((x_dash, y_dash, z_dash)))
    final_temp.append(a)
    labels_temp.append(temptemptemp.head(1)["tac"])
  final.append(np.array(final_temp))
  labels.append(np.array(labels_temp))
  # print(final)
  
  # break
# print(np.array(final,dtype=object).shape)


In [134]:
# [-0.1331,  0.026 ,  0.1165, -0.0039, -0.1326, -0.1095,  0.1165,
        #  0.1467, -0.1494, -0.0233]
a.shape

(200, 3)

In [135]:
final_arr = np.asarray(final).astype('float32')
labels_arr = np.asarray(labels).astype('float32')
print(final_arr.shape)
print(labels_arr.shape)

(1, 30735, 200, 3)
(1, 30735, 1)


In [136]:
final_arr_reshape = np.reshape(final_arr, (30735, 200, 3)) 
labels_arr_reshape = np.reshape(labels_arr, (30735,1))
final_arr_reshape.shape, labels_arr_reshape.shape 

((30735, 200, 3), (30735, 1))

In [137]:
# from TF_Model import Drunk
# drunk = Drunk()

# drunk.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), \
#                    optimizer=tf.keras.optimizers.Adam(0.03), \
#                    metrics=[tf.keras.metrics.Accuracy()]) 

# drunk.build((30735, 600))

## Baseline Model 
#### 3 hidden layers (32 nodes, RELU, 32 nodes, RELU, 16 nodes, RELU)  
#### output layer uses sigmoid activation

In [139]:
base_model_data = np.reshape(final_arr, (30735, 200, 3)) 
base_model_labels = np.reshape(labels_arr, (30735,1))

batch_size = 32
epochs = 50
# prep_data = pd.read_csv('../data/good_again_bhas.csv')
flatten = tf.keras.layers.Flatten()
fc_layer1 = tf.keras.layers.Dense(units=32, activation = 'relu')
fc_layer2 = tf.keras.layers.Dense(units=32, activation = 'relu')
fc_layer3 = tf.keras.layers.Dense(units=16, activation = 'relu')
fc_layer4 = tf.keras.layers.Dense(units=1, activation = 'sigmoid') 
base_model = tf.keras.Sequential([flatten, fc_layer1, fc_layer2, fc_layer3, fc_layer4])

base_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), \
                   optimizer=tf.keras.optimizers.Adam(0.03), \
                   metrics=[tf.keras.metrics.Accuracy()]) 

base_model.build((30735, 200, 3))
base_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_17 (Flatten)        (30735, 600)              0         
                                                                 
 dense_40 (Dense)            (30735, 32)               19232     
                                                                 
 dense_41 (Dense)            (30735, 32)               1056      
                                                                 
 dense_42 (Dense)            (30735, 16)               528       
                                                                 
 dense_43 (Dense)            (30735, 1)                17        
                                                                 
Total params: 20,833
Trainable params: 20,833
Non-trainable params: 0
_________________________________________________________________


In [140]:
base_model.fit(base_model_data, base_model_labels, epochs = 10,
            #    batch_size = 32, 
               verbose=1) 

Epoch 1/10
961/961 [==============================] - 1s 699us/step - loss: 0.6224 - accuracy: 0.0000e+00
Epoch 2/10
961/961 [==============================] - 1s 622us/step - loss: 0.6050 - accuracy: 0.0000e+00
Epoch 3/10
961/961 [==============================] - 1s 606us/step - loss: 0.6035 - accuracy: 0.0000e+00
Epoch 4/10
961/961 [==============================] - 1s 636us/step - loss: 0.5996 - accuracy: 3.2536e-05
Epoch 5/10
961/961 [==============================] - 1s 587us/step - loss: 0.5934 - accuracy: 0.0000e+00
Epoch 6/10
961/961 [==============================] - 1s 586us/step - loss: 0.6034 - accuracy: 1.9522e-04
Epoch 7/10
961/961 [==============================] - 1s 588us/step - loss: 0.5923 - accuracy: 0.0000e+00
Epoch 8/10
961/961 [==============================] - 1s 640us/step - loss: 0.5930 - accuracy: 0.0000e+00
Epoch 9/10
961/961 [==============================] - 1s 684us/step - loss: 0.5889 - accuracy: 0.0000e+00
Epoch 10/10
961/961 [=========================

In [141]:
loss, accuracy = base_model.evaluate(base_model_data, base_model_labels,
                                    #  batch_size = batch_size, 
                                     verbose=0)

round(loss,4),accuracy

(0.5921, 0.0)

## CNN Base Model

In [124]:
# import tensorflow as tf

# class MyModel(tf.keras.Model):

#   def __init__(self):
#     super().__init__()
#     flatten = tf.keras.layers.Flatten()
#     conv_layer1 = tf.keras.layers.Conv1D(filters = 64, kernel_size = 3)
#     conv_layer2 = tf.keras.layers.Conv1D(filters = 64, kernel_size = 3)
#     dropout = tf.keras.layers.Dropout(0.5)
#     max_pooling = tf.keras.layers.MaxPool1D(pool_size=2)
#     # fc - fully connected layer
#     fc_layer = tf.keras.layers.Dense(units=64, activation = 'relu')
#     fc_layer2 = tf.keras.layers.Dense(units=1, activation = 'softmax') 
#     self.base_model = tf.keras.Sequential([
#                                   conv_layer1, 
#                                   conv_layer2, 
#                                   dropout, 
#                                   max_pooling, 
#                                   # flatten, 
#                                   fc_layer, 
#                                   fc_layer2
#                                 ])


#   def call(self, inputs, training=True):
#     return self.base_model(inputs)
    
#     # if training:
#     #   x = self.dropout(x, training=training)
#     # return self.dense2(x)

# model = MyModel()

In [125]:
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), \
#                    optimizer=tf.keras.optimizers.Adam(0.001), \
#                    metrics=[tf.keras.metrics.Accuracy()]) 

# model.build((30735, 3, 200))
# model.summary()

In [126]:
# model.fit(final_arr_reshape, labels_arr_reshape, epochs = 100,
#             #    batch_size = batch_size, 
#                verbose=1) 

In [143]:
batch_size = 32
epochs = 50
# prep_data = pd.read_csv('../data/good_again_bhas.csv')
flatten = tf.keras.layers.Flatten()
conv_layer1 = tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, padding='SAME')
conv_layer2 = tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, padding='SAME')
dropout = tf.keras.layers.Dropout(0.5)
max_pooling = tf.keras.layers.MaxPool1D(pool_size=2)
# fc - fully connected layer
fc_layer = tf.keras.layers.Dense(units=64, activation = 'relu')
fc_layer2 = tf.keras.layers.Dense(units=1, activation = 'softmax') 
base_model = tf.keras.Sequential([
                                  conv_layer1, 
                                  conv_layer2, 
                                  dropout, 
                                  max_pooling, 
                                  flatten, 
                                  fc_layer, 
                                  fc_layer2
                                ])

base_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), \
                   optimizer=tf.keras.optimizers.Adam(0.001), \
                   metrics=[tf.keras.metrics.Accuracy()]) 

In [144]:
# loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# optimizer = tf.keras.optimizers.Adam(0.001)

# for e in range(0, 5000):
#     with tf.GradientTape() as tape:
#         preds = base_model(final_arr_reshape)
#         print(preds)
        
#         # loss = loss_func(y_true=labels_arr_reshape, y_pred=preds)
#         # acc = np.sum(np.equal(labels, preds)) / 30735
#     #     print(acc)
#     # gradients = tape.gradient(loss, base_model.trainable_variables)
#     # optimizer.apply_gradients(zip(gradients, base_model.trainable_variables))
#     # print(f"Epoch: {e} | LOSS : {loss} | acc {acc}")

# # loss
            
# # total_loss += loss
# # gradients = tape.gradient(loss, model.trainable_variables)
# #     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
# #     return total_loss


In [145]:
base_model.build((30735, 200, 3))

In [146]:
base_model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_26 (Conv1D)          (30735, 200, 64)          1024      
                                                                 
 conv1d_27 (Conv1D)          (30735, 200, 64)          20544     
                                                                 
 dropout_14 (Dropout)        (30735, 200, 64)          0         
                                                                 
 max_pooling1d_13 (MaxPoolin  (30735, 100, 64)         0         
 g1D)                                                            
                                                                 
 flatten_18 (Flatten)        (30735, 6400)             0         
                                                                 
 dense_44 (Dense)            (30735, 64)               409664    
                                                     

In [147]:
np.unique(labels_arr, return_counts=True)

(array([0., 1.], dtype=float32), array([11229, 19506]))

In [148]:
print(all_labels_df.shape)
all_labels_df['tac'].value_counts()


(1225727, 2)


tac
1    778034
0    447693
Name: count, dtype: int64

In [149]:
base_model.fit(final_arr_reshape, labels_arr_reshape, epochs = 10,
               batch_size = batch_size, 
               verbose=1) 

# base_model.fit(base_model_data, base_model_labels, epochs = 100,
#             #    batch_size = batch_size, 
#                verbose=1) 

Epoch 1/10
961/961 [==============================] - 8s 8ms/step - loss: 0.6288 - accuracy: 0.6347
Epoch 2/10
961/961 [==============================] - 9s 9ms/step - loss: 0.6155 - accuracy: 0.6347
Epoch 3/10
961/961 [==============================] - 8s 9ms/step - loss: 0.6082 - accuracy: 0.6347
Epoch 4/10
961/961 [==============================] - 9s 9ms/step - loss: 0.6040 - accuracy: 0.6347
Epoch 5/10
961/961 [==============================] - 8s 9ms/step - loss: 0.5985 - accuracy: 0.6347
Epoch 6/10
961/961 [==============================] - 8s 9ms/step - loss: 0.5951 - accuracy: 0.6347
Epoch 7/10
961/961 [==============================] - 8s 8ms/step - loss: 0.5896 - accuracy: 0.6347
Epoch 8/10
961/961 [==============================] - 8s 8ms/step - loss: 0.5842 - accuracy: 0.6347
Epoch 9/10
961/961 [==============================] - 9s 9ms/step - loss: 0.5761 - accuracy: 0.6347
Epoch 10/10
961/961 [==============================] - 9s 9ms/step - loss: 0.5735 - accuracy: 0.6347

In [150]:
# loss, accuracy = base_model.evaluate(final_arr_reshape, labels_arr_reshape,
#                                     #  batch_size = batch_size, 
#                                      verbose=0)

loss, accuracy = base_model.evaluate(base_model_data, base_model_labels,
                                    #  batch_size = batch_size, 
                                     verbose=0)

round(loss,4), accuracy

(0.5554, 0.6346510648727417)

## MLP Classifier

In [155]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
clf = MLPClassifier(solver='adam', random_state=1)

In [156]:
final_arr_mlp = np.reshape(final_arr, (30735, 600))
labels_arr_mlp = np.reshape(labels_arr, (30735,1))

clf.fit(final_arr_mlp, labels_arr_mlp)
# print('Accuracy ', accuracy_score(y_test, clf.predict(x_test)))

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(random_state=1)

In [157]:
print('Accuracy ', accuracy_score(labels_arr_mlp, clf.predict(final_arr_mlp)))

Accuracy  0.8023100699528225


## LSTM Model 
#### LSTM Layer (128 units), Dropout layer (p=0.5), Dense layer (128 units)

In [162]:
base_model_data = np.reshape(final_arr, (30735, 200, 3)) 
base_model_labels = np.reshape(labels_arr, (30735,1))

batch_size = 32
epochs = 50
# prep_data = pd.read_csv('../data/good_again_bhas.csv')
flatten = tf.keras.layers.Flatten()
lstm_layer = tf.keras.layers.LSTM(units=128)
dropout_layer = tf.keras.layers.Dropout(0.5)
fc_layer = tf.keras.layers.Dense(units=128, activation='relu')
fc_layer2 = tf.keras.layers.Dense(units=1, activation='softmax')
lstm_model = tf.keras.Sequential([flatten, lstm_layer, dropout_layer,
                                # flatten, 
                                fc_layer, fc_layer2])

lstm_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), \
                   optimizer=tf.keras.optimizers.Adam(0.03), \
                   metrics=[tf.keras.metrics.Accuracy()]) 

lstm_model.build((30735, 200, 3))
lstm_model.summary()

ValueError: Input 0 of layer "lstm_3" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (30735, 600)

In [161]:
lstm_model.fit(base_model_data, base_model_labels, epochs = 100,
            #    batch_size = batch_size, 
               verbose=1) 

Epoch 1/100


ValueError: in user code:

    File "/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [ ]:
loss, accuracy = lstm_model.evaluate(base_model_data, base_model_labels,
                                    #  batch_size = batch_size, 
                                     verbose=1)

round(loss,4),accuracy

961/961 [==============================] - 2s 1ms/step - loss: 0.5258 - accuracy: 0.6347


(0.5258, 0.6346510648727417)

Bad pipe message: %s [b'\x02P\xcep\xd4\x9d\xebA\x8a7+\x9a+\xc0\xbe\x08\xa8\x9d 2\xd4\xb9\xf5;\xc4\xe0\xde\xdeeKk\x1e\x01\x8e\xc3\x86\xb5\x03\x0e\x8e\x83\x82@\xfe\x06\x863Z\x91V\x87\x00 **\x13\x01\x13\x02\x13\x03\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x01\x93\xba\xba\x00\x00\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\x1b\x00\x03\x02\x00\x02\x00\x17\x00\x00\x00\n\x00\n\x00\x08\x8a\x8a\x00\x1d\x00\x17\x00\x18\xff\x01\x00\x01\x00Di\x00\x05\x00\x03\x02h2\x00\r\x00\x12\x00\x10\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x003\x00+\x00)\x8a\x8a\x00\x01\x00\x00\x1d\x00 \x15\xbb\xcc+\xdf\xd6R\xc2\xf4i\x96\xd2{x^\xe4#\xd4>\x9cq\xb6[']
Bad pipe message: %s [b'\x8e\x16\xea\xe5\x02\x89\x08\x00+\x00\x07\x06\xba\xba\x03\x04\x03\x03\x00\x0b\x00\x02\x01\x00\x00\x10\x00\x0e\x00\x0c\x02h2\x08http/1.1\x00-\x00\x02\x01\x01\x00\x12\x00\x00\x00#\x00\x00\xfa\xfa\x00\x01\x00\x00\x15\x00\xe0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0